In [ ]:
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
import requests
import io

In [ ]:
#url = "REPLACE-THIS-WITH-THE-URL-OF-THE-CSV-FILE" 
#download = requests.get(url).content
df  = pd.read_csv('heart.csv')
df.head()

In [ ]:
df['age'].unique()

In [ ]:
df.isna().sum()

In [ ]:
plt.figure(figsize=(15,15))
hm = sns.heatmap(df.corr(), xticklabels=True,yticklabels=True,annot=True)
plt.show()

In [ ]:
df.head()

In [ ]:
mms = MinMaxScaler()
df[['trtbps','chol','thalachh','oldpeak']] = mms.fit_transform(df[['trtbps','chol','thalachh','oldpeak']])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#for tf model
y = df.pop('output')

In [ ]:
#for tf model
xtrain, xtest, ytrain, ytest = train_test_split(df,y, test_size=0.25, random_state=0)
print([xtrain.shape,ytrain.shape,xtest.shape,ytest.shape])

In [ ]:
categorical_columns = ['sex','cp','fbs','restecg','exng','slp','caa','thall']
numeric_columns = ['trtbps','chol','thalachh','oldpeak']
feat_cols = []
for x in categorical_columns:
    vocabulary = df[x].unique()
    feat_cols.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(x,vocabulary)))
for x in numeric_columns:
    feat_cols.append(tf.feature_column.numeric_column(x,dtype = tf.float32))
age = tf.feature_column.numeric_column("age")
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feat_cols.append(age_buckets)
print(feat_cols)

In [ ]:
#tf model sathi
def make_input_function(data_df, label_df, num_epochs = 20, shuffle = True, batch_size = 32):
    def input_func():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_func
train_input_func = make_input_function(xtrain, ytrain)
test_input_func = make_input_function(xtest, ytest, num_epochs=1, shuffle=False)

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feat_cols)
linear_est = linear_est.train(train_input_func)
result = linear_est.evaluate(test_input_func)
print(result['accuracy'])

In [ ]:
#pickle.dump(linear_est, open('lin_est.pkl', 'wb'))
joblib.dump(linear_est, 'lin_est.pkl')

In [ ]:
#Ann model sathi
def df_to_dataset(dataframe, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('output')
    return tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)).shuffle(buffer_size = len(dataframe)).batch(batch_size)
train, test = train_test_split(df, test_size=0.2, random_state=32)
train_input_func = df_to_dataset(train)
test_input_func = df_to_dataset(test)

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feat_cols)

In [ ]:
model = tf.keras.models.Sequential([
    feature_layer,
    tf.keras.layers.Dense(units = 128, activation = 'relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_input_func, validation_data=test_input_func, batch_size=64, epochs=2, use_multiprocessing=True)

In [ ]:
model.evaluate(test_input_func)

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:
predictions = model.predict(test_input_func)
print(predictions)